# Tutorial 04 - Optimizing Force Fields

In this tutorial we will be using the OpenFF Evaluator framework in combination with the fantastic [ForceBalance](
https://github.com/leeping/forcebalance) software to optimize a molecular force field against the physical property data 
set we created in the [first tutorial](tutorial01.ipynb).

ForceBalance offers a suite of tools for optimizing molecular force fields against a set of target data. Perhaps one of 
the most fundamental targets to fit against is experimental physical property data. Physical property data has been used 
extensively for decades to inform the values of non-bonded Van der Waals (VdW) interaction parameters (often referred to 
as Lennard-Jones parameters).

ForceBalance is seamlessly integrated with the evaluator framework, using it to evaluate the deviations between
target experimentally measured data points and those evaluated using the force field being optimized (as well as the 
gradient of those deviations with respect to the force field parameters being optimized).

The tutorial will cover:

- setting up the input files and directory structure required by ForceBalace.
- setting up an EvaluatorServer for ForceBalance to connect to.
- running ForceBalance using those input files.
- extracting and plotting a number of statistics output during the optimization.

*Note: If you are running this tutorial in google colab you will need to run a setup script instead of following the 
installation instructions:*

In [ ]:
# !wget https://raw.githubusercontent.com/openforcefield/openff-evaluator/master/docs/tutorials/colab_setup.ipynb
# %run colab_setup.ipynb

*For this tutorial make sure that you are using a GPU accelerated runtime.*

For the sake of clarity all warnings will be disabled in this tutorial:

In [ ]:
import warnings
warnings.filterwarnings('ignore')
import logging
logging.getLogger("openforcefield").setLevel(logging.ERROR)

We will also enable time-stamped logging to help track the progress of our calculations:

In [ ]:
from evaluator.utils import setup_timestamp_logging
setup_timestamp_logging()

## Setting up the ForceBalance Inputs

We will begin by creating the directory structure required by ForceBalance before populating it with the required input 
files.

### Creating the Directory Structure

We will need to create a directory to store the starting force field parameters in:

In [ ]:
!mkdir forcefield

and one to store the input parameters for our 'fitting target' - in this case a data set of physical properties:

In [ ]:
!mkdir -p targets/pure_data

### Storing the Initial Force Field Parameters

For this example optimization we will use the OpenFF Parsley force field as the starting parameters. This can be 
downloaded directly from github:

In [ ]:
!cd forcefield
!wget https://raw.githubusercontent.com/openforcefield/openforcefields/master/openforcefields/offxml/openff-1.0.0.offxml
!cd ../

*Note: The force field parameters are stored in the OpenFF SMIRNOFF format, more information about which can be 
[found here](https://open-forcefield-toolkit.readthedocs.io/en/0.6.0/smirnoff.html).*

### Creating the Main Input File

Next, we will create the main ForceBalance input file. For the sake of brevity for this tutorial a default input file
which ships with this framework will be used:

In [ ]:
from evaluator.utils import get_data_filename
input_file_path = get_data_filename("tutorials/tutorial04/optimize.in")

# Copy the input file into our directory structure
from shutil import copyfile
shutil.copyfile(input_file_path, "optimize.in")

While there are many options that can be set within this file, the main options of interest for our purposes appear
at the bottom of the file:

In [ ]:
!tail -n 6 optimize.in

Here we have specified that we wish to create a new ForceBalance `Evaluator_SMIRNOFF` target called `pure_data` 
(corresponding to the name of the directory we created in the earlier step).
 
The main input to this target is the file path to an `options.json` file - it is this file which will specify all the
options which should be used when ForceBalance requests that our target data set be estimated using the current sets
of force field parameters.

We will create this file in the `targets/pure_data` directory later in this section.

### Defining the Training Data Set

Before defining the estimation options, we will store the data set which we will be training the force field against
in the target directory created earlier:

In [ ]:
# For convenience we will use the copy shipped with the framework
data_set_file_path = get_data_filename("tutorials/tutorial01/filtered_data_set.json")

# The data set needs to be stored in our targets folder:
shutil.copyfile(data_set_file_path, "targets/pure_data/training_set.json")

The data set is the JSON serialized representation of the `PhysicalPropertyDataSet` we created during the [first 
tutorial](tutorial01.ipynb).

### Defining the Estimation Options

The final step before we can start the optimization is to create the set of options which will govern how our data set
is estimated using the Evaluator framework.
 
These options will be stored in an `Evaluator_SMIRNOFF` object:

In [ ]:
from forcebalance.evaluator import Evaluator_SMIRNOFF

# Create the force balance options object
target_options = Evaluator_SMIRNOFF.OptionsFile()
target_options.estimation_options = evaluator_options

This object exposes both a set of ForceBalance specific options, as well as the set of Evaluator options. 

The ForceBalance specific options allow us to define how each type of property will contribute to the optimization
objective function (the value which we are trying to minimize):

$$\Delta(\theta) = \sum^N_n \dfrac{weight_n}{M_n} \sum^{M_n}_m \left(\dfrac{y_m^{ref} - y_m(\theta)}{denominator_{n}}\right)^2$$ 

where $N$ is the number of types of properties (e.g. density, enthalpy of vaporization, etc.), $M_n$ is the number of
data points of type $n$, $y_m^{ref}$ is the experimental value of data point $m$ and $y_m(\theta)$ is the estimated 
value of data point $m$ using the current force field parameters

In particular, the options object allows us to specify both an amount to scale each type of properties contribution to 
the objective function by ($weight_n$), and the amount to scale the difference between the experimental and estimated
properties ($denominator_n$):

In [ ]:
from evaluator import unit

target_options.weights = {
    "Density": 1.0,
    "EnthalpyOfVaporization": 1.0
}
target_options.denominators = {
    "Density": 30.0 * unit.kilogram / unit.meter ** 3,
    "EnthalpyOfVaporization": 3.0 * unit.kilojoule / unit.mole
}
 

 
where here we have chosen values that ensure that both types of properties contribute roughly the same to the total
objective function.

The Evaluator specific options correspond to a standard `RequestOptions` object:

In [ ]:
# Create the options which evaluator should use.
evaluator_options = RequestOptions()
# Choose which calculation layers to make available.
evaluator_options.calculation_layers = ["SimulationLayer"]

These options allow us to control exactly how each type of property should be estimated, which calculation approaches
should be used and more. Here we just use the default options, however more information about the different choices
exposed can be [found here](../gettingstarted/client.rst).

We must additionally specify the name of the data set object which we wish to fit against:

In [ ]:
# Set the path to the data set
target_options.data_set_path = "data_set.json"

And that's the options created! We will finish off by serializing the options into our target directory:

In [ ]:
# Save the options to file.
with open("targets/pure_data/options.json", "w") as file:
    file.write(target_options.to_json())

## Launching an Evaluator Server

With the force balance options created, we can now move onto launching the `EvaluatorServer` which ForceBalance will 
call out to when it needs the data set to be evaluated:

In [ ]:
# Launch the calculation backend which will distribute any calculations.
from evaluator.backends import ComputeResources
from evaluator.backends.dask import DaskLocalCluster

calculation_backend = DaskLocalCluster(
    number_of_workers=1,
    resources_per_worker=ComputeResources(
        number_of_threads=1, 
        number_of_gpus=0, 
        preferred_gpu_toolkit=ComputeResources.GPUToolkit.CUDA
    ),
)
calculation_backend.start()

# Launch the server object which will listen for estimation requests and schedule any 
# required calculations.
from evaluator.server import EvaluatorServer

evaluator_server = EvaluatorServer(calculation_backend=calculation_backend)
evaluator_server.start(asynchronous=True)

We will not go into the details of this here as this was already covered in the [second tutorial](tutorial02.ipynb)

## Running ForceBalance

With the inputs created and an Evaluator server spun up, we are finally ready to run the optimization! If everything
went well so far, this can be accomplished with a single command:

In [ ]:
!ForceBalance optimize.in

If everything went well ForceBalance should exit cleanly, and will have stored out newly optimized force field in the
`results` directory.

In [ ]:
!ls results/*